# Interactive Python Notebook of Hodgekin-Huxley Model

In [1]:
#%pip install ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
plt.style.use('ggplot')
from scipy.integrate import odeint

my_layout = widgets.Layout()

In [2]:
def hodgkinHuxley(yy,t, p):

	#Name the variables
	V = yy[0]
	n = yy[1]
	m = yy[2]
	h = yy[3]
	
	#Name the parameters
	C_m = p[0]
	V_Na = p[1]
	V_K = p[2]
	V_l = p[3]
	g_Na = p[4]
	g_K = p[5]
	g_l = p[6]
	
	#The injected current? ? ? ? 
	I = p[7]
	
	#Auxiliary quantities	
	alpha_n = (0.01*(10-V)) / (np.exp((10-V)/10) -1) 
	beta_n = 0.125 * np.exp(-V/80)	
	alpha_m = (0.1 *(25-V)) / (np.exp((25-V)/10) -1) 
	beta_m  = 4 * np.exp(-V/18)
	alpha_h = 0.07 * np.exp(-V/20)
	beta_h = 1 / (np.exp((30-V)/10) +1) 
	
	# HH model

	V_dot = (-g_K*n**4*(V-V_K) - g_Na*m**3*h*(V-V_Na) - g_l*(V-V_l) + I)/C_m
	n_dot = alpha_n * (1-n) - beta_n *n
	m_dot = alpha_m * (1-m) - beta_m *m
	h_dot = alpha_h * (1-h) - beta_h *h
	
	
	return [V_dot, n_dot, m_dot, h_dot]

In [5]:
#Define initial conditions for the state variables
y0 = [0.0,0.0,0.0,0.0]

#Define time interval and spacing for the solutions
t = np.arange(0,100,0.01)

#Define fixed parameters pp. 520 (HH1952 paper)
C_m = 1

E_Na = 50.
E_K = -77.
E_l = -54.387
RP = -65.
V_Na = E_Na - RP
V_K = E_K - RP
V_l = E_l - RP
g_Na = 120
g_K = 36
g_l = 0.3
#no current? 
# I = -20

In [6]:
@widgets.interact(
    I=widgets.FloatSlider(50., min=-20., max=150., step=1.,
                               layout=my_layout),
                               
)
def HH_plot(I):
    plt.figure(figsize=(10, 5))
    plt.ylabel("Volts (mV)")
    plt.xlabel("Time (msec)")
    plt.suptitle("HH-model Dynamics")
    #Pack the parameters in a single vector
    p = [C_m,V_Na, V_K,V_l,g_Na,g_K,g_l,I] 
    #Call the integrator 
    y = odeint(hodgkinHuxley, y0, t, args=(p,)) 
    plt.plot(t,y[:,0])

interactive(children=(FloatSlider(value=50.0, description='I', max=150.0, min=-20.0, step=1.0), Output()), _do…